In [1]:
from tqdm.notebook import trange, tqdm
from typing import TypedDict, NamedTuple, Literal, NotRequired
from tba_types import Award, Event
from tba_session import session, ROOT_URL

In [2]:
TEAM_PAGES = 22

In [3]:
ALL_TEAM_KEYS: list[str] = []

for i in trange(TEAM_PAGES, desc="Getting all teams", unit="page"):
    ALL_TEAM_KEYS.extend(
        session.get(f"{ROOT_URL}/teams/{i}/keys").json()
    )

Getting all teams:   0%|          | 0/22 [00:00<?, ?page/s]

In [4]:
TEAM_EVENT_AWARDS: dict[str, dict[str, list[Award]]] = {}
EVENT_KEY_MAP: dict[str, str] = {}
EVENT_MAP: dict[str, Event] = {}

for team_key in tqdm(
    ALL_TEAM_KEYS, desc="Getting team events", unit="team", leave=False
):
    TEAM_EVENT_AWARDS[team_key] = {}
    team_events: list[Event] = session.get(
        f"{ROOT_URL}/team/{team_key}/events"
    ).json()
    for event in sorted(team_events, key=lambda event: event["start_date"]):
        EVENT_MAP[event["key"]] = event
        if (
            not 0 <= event["event_type"] <= 5
            or event["year"] == 2021
            or "cancelled" in event["name"].lower()
        ):
            continue
        EVENT_KEY_MAP[event["key"]] = (
            event["key"]
            if event["parent_event_key"] is None
            else event["parent_event_key"]
        )
        TEAM_EVENT_AWARDS[team_key][EVENT_KEY_MAP[event["key"]]] = []
    team_awards: list[Award] = session.get(
        f"{ROOT_URL}/team/{team_key}/awards"
    ).json()
    for award in team_awards:
        if (
            award["event_key"] not in EVENT_KEY_MAP
            or 1 <= award["award_type"] <= 8
            or award["award_type"] == 14
            or award["award_type"] == 39
            or award["award_type"] == 68
            # or not 3 <= award["award_type"] <= 5
        ):
            continue
        if EVENT_KEY_MAP[award["event_key"]] not in TEAM_EVENT_AWARDS[team_key]:
            print(team_key, award)
            TEAM_EVENT_AWARDS[team_key][EVENT_KEY_MAP[award["event_key"]]] = []
        TEAM_EVENT_AWARDS[team_key][EVENT_KEY_MAP[award["event_key"]]].append(award)

Getting team events:   0%|          | 0/8831 [00:00<?, ?team/s]

frc203 {'award_type': 9, 'event_key': '2019mrcmp', 'name': 'Regional Engineering Inspiration Award', 'recipient_list': [{'awardee': None, 'team_key': 'frc203'}, {'awardee': None, 'team_key': 'frc6016'}], 'year': 2019}
frc568 {'award_type': 9, 'event_key': '2018pncmp', 'name': 'Regional Engineering Inspiration Award', 'recipient_list': [{'awardee': None, 'team_key': 'frc2980'}, {'awardee': None, 'team_key': 'frc568'}], 'year': 2018}
frc568 {'award_type': 9, 'event_key': '2019pncmp', 'name': 'Regional Engineering Inspiration Award', 'recipient_list': [{'awardee': None, 'team_key': 'frc568'}, {'awardee': None, 'team_key': 'frc6831'}], 'year': 2019}
frc2980 {'award_type': 9, 'event_key': '2018pncmp', 'name': 'Regional Engineering Inspiration Award', 'recipient_list': [{'awardee': None, 'team_key': 'frc2980'}, {'awardee': None, 'team_key': 'frc568'}], 'year': 2018}
frc3574 {'award_type': 9, 'event_key': '2017pncmp', 'name': 'Regional Engineering Inspiration Award', 'recipient_list': [{'awar

In [5]:
def longest_award_streak(
    unique: bool = False, identical: bool = False
) -> dict[str, tuple[int, dict[str, list[Award]]]]:
    if unique and identical:
        raise ValueError("unique and identical cannot be both True")

    longest_streak: dict[str, tuple[int, dict[str, list[Award]]]] = {}
    for team_key, events in tqdm(TEAM_EVENT_AWARDS.items()):
        longest_streak[team_key] = (0, {})
        items = list(events.items())
        start = -1
        for end in range(len(items)):
            if not items[end][1]:
                start = -1
                continue
            elif start == -1:
                start = end
            else:
                while (
                    (not identical and not unique and not items[start])
                    or (
                        unique
                        and all(
                            award_type
                            in set(
                                award["award_type"]
                                for awards in items[start:end]
                                for award in awards[1]
                            )
                            for award_type in map(
                                lambda x: x["award_type"], items[end][1]
                            )
                        )
                    )
                    or (
                        identical
                        and not any(
                            all(
                                award_type
                                in set(award["award_type"] for award in item[1])
                                for item in items[start:end]
                            )
                            for award_type in map(
                                lambda x: x["award_type"], items[end][1]
                            )
                        )
                    )
                ):
                    start += 1
            if end - start + 1 > longest_streak[team_key][0]:
                longest_streak[team_key] = (
                    end - start + 1,
                    dict(items[start : end + 1]),
                )
    return longest_streak

In [6]:
SENTINEL = 200
prev_length = SENTINEL
min_length = 5
should_be_active = False
for team_key, vals in sorted(
    longest_award_streak(False, True).items(), key=lambda x: (x[1][0], 20000 - int(x[0][3:])), reverse=True
):
    length, awards = vals
    if length < min_length:
        print("[/details]")
        break
    if should_be_active and not (
        list(awards.keys())[-1] == list(TEAM_EVENT_AWARDS[team_key].keys())[-1]
        and list(awards.keys())[-1].startswith("2025")
    ):
        continue
    if length < prev_length:
        if prev_length != SENTINEL:
            print("[/details]")
        print(f'[details="{length} awards"]')
        prev_length = length
    print(f"## [{team_key[3:]}](https://www.thebluealliance.com/team/{team_key[3:]})")
    for event_key, awards in awards.items():
        print(
            f"### [{EVENT_MAP[event_key]["year"]} {EVENT_MAP[event_key]["name"]}](https://www.thebluealliance.com/event/{event_key})"
        )
        for award in awards:
            print(award["name"])

  0%|          | 0/8831 [00:00<?, ?it/s]

[details="8 awards"]
## [193](https://www.thebluealliance.com/team/193)
### [2023 FMA District Mount Olive Event](https://www.thebluealliance.com/event/2023njfla)
Imagery Award in honor of Jack Kamen
### [2023 FMA District Warren Hills Event](https://www.thebluealliance.com/event/2023njwas)
Imagery Award in honor of Jack Kamen
### [2023 FIRST Mid-Atlantic District Championship](https://www.thebluealliance.com/event/2023mrcmp)
Imagery Award in honor of Jack Kamen
### [2024 FMA District Mount Olive Event](https://www.thebluealliance.com/event/2024njfla)
Imagery Award in honor of Jack Kamen
### [2024 FMA District Warren Hills Event](https://www.thebluealliance.com/event/2024njwas)
Imagery Award in honor of Jack Kamen
### [2025 FMA District Mount Olive Event](https://www.thebluealliance.com/event/2025njfla)
Imagery Award in honor of Jack Kamen
### [2025 FMA District Centennial Event](https://www.thebluealliance.com/event/2025pawar)
Imagery Award in honor of Jack Kamen
### [2025 FIRST Mid-A

In [7]:
team = ""
event = ""
num_awards = 0
for team_key in TEAM_EVENT_AWARDS:
    for event_key in TEAM_EVENT_AWARDS[team_key]:
        if len(TEAM_EVENT_AWARDS[team_key][event_key]) >= num_awards:
            num_awards = len(TEAM_EVENT_AWARDS[team_key][event_key])
            team = team_key
            event = event_key
            print(
                f"[{team_key[3:]}](https://www.thebluealliance.com/team/{team_key[3:]}/{event_key[:4]}) won {num_awards} awards at [{event_key[:4]} {EVENT_MAP[event_key]["name"]}](https://www.thebluealliance.com/event/{event_key})",
                "("
                + ", ".join(
                    map(
                        lambda x: x["name"]
                        + (
                            (
                                " ("
                                + ", ".join(
                                    a["awardee"]
                                    for a in x["recipient_list"]
                                    if a["team_key"] == team_key
                                )
                                + ")"
                            )
                            if [r for r in x["recipient_list"] if r["awardee"]]
                            else ""
                        ),
                        TEAM_EVENT_AWARDS[team_key][event_key],
                    )
                )
                + ")",
            )

[1](https://www.thebluealliance.com/team/1/1997) won 0 awards at [1997 Motorola Midwest Regional](https://www.thebluealliance.com/event/1997il) ()
[1](https://www.thebluealliance.com/team/1/1997) won 0 awards at [1997 National Championship](https://www.thebluealliance.com/event/1997cmp) ()
[1](https://www.thebluealliance.com/team/1/1998) won 1 awards at [1998 Great Lakes Regional](https://www.thebluealliance.com/event/1998mi) (Quality)
[1](https://www.thebluealliance.com/team/1/1998) won 1 awards at [1998 National Championship](https://www.thebluealliance.com/event/1998cmp) (Best Play of the Day Award)
[1](https://www.thebluealliance.com/team/1/1999) won 1 awards at [1999 Motorola Midwest Regional](https://www.thebluealliance.com/event/1999il) (Creativity)
[1](https://www.thebluealliance.com/team/1/2000) won 2 awards at [2000 Great Lakes Regional](https://www.thebluealliance.com/event/2000mi) (Creativity, Best Offensive Round)
[1](https://www.thebluealliance.com/team/1/2000) won 2 awar

In [ ]:
if "get_ipython" not in globals():
    import json
    import pathlib

    output_folder = pathlib.Path(__file__).parent / "build"

    config = {
        "columns": [
            {
                "name": "team",
                "type": "team",
                "displayName": "Team",
            },
            {
                "name": "length",
                "type": "integer",
                "displayName": "Streak Length",
            },
            {
                "name": "event_start",
                "type": "event",
                "displayName": "Start Event",
            },
            {
                "name": "event_end",
                "type": "event",
                "displayName": "End Event",
            },
            {
                "name": "active",
                "type": "boolean",
                "displayName": "Active",
            },
        ],
        "defaultSort": [{"column": "length", "direction": -1}],
        "primaryKey": "team",
    }

    with (output_folder / "unique-award-streaks.config.json").open("w") as f:
        json.dump({**config, "title": "Longest Unique Award Streaks"}, f, indent=None)
    with (output_folder / "unique-award-streaks.json").open("w") as f:
        json.dump(
            [
                {
                    "team": team_key,
                    "length": vals[0],
                    "event_start": list(vals[1].keys())[0],
                    "event_end": list(vals[1].keys())[-1],
                    "active": list(vals[1].keys())[-1]
                    == list(TEAM_EVENT_AWARDS[team_key].keys())[-1]
                    and list(vals[1].keys())[-1].startswith("2025"),
                }
                for team_key, vals in longest_award_streak(True, False).items()
                if list(vals[1].keys())
            ],
            f,
            indent=None,
        )
    with (output_folder / "identical-award-streaks.config.json").open("w") as f:
        json.dump(
            {**config, "title": "Longest Identical Award Streaks"}, f, indent=None
        )
    with (output_folder / "identical-award-streaks.json").open("w") as f:
        json.dump(
            [
                {
                    "team": team_key,
                    "length": vals[0],
                    "event_start": list(vals[1].keys())[0],
                    "event_end": list(vals[1].keys())[-1],
                    "active": list(vals[1].keys())[-1]
                    == list(TEAM_EVENT_AWARDS[team_key].keys())[-1]
                    and list(vals[1].keys())[-1].startswith("2025"),
                }
                for team_key, vals in longest_award_streak(False, True).items()
                if list(vals[1].keys())
            ],
            f,
            indent=None,
        )
    with (output_folder / "award-streaks.config.json").open("w") as f:
        json.dump({**config, "title": "Longest Award Streaks"}, f, indent=None)
    with (output_folder / "award-streaks.json").open("w") as f:
        json.dump(
            [
                {
                    "team": team_key,
                    "length": vals[0],
                    "event_start": list(vals[1].keys())[0],
                    "event_end": list(vals[1].keys())[-1],
                    "active": list(vals[1].keys())[-1]
                    == list(TEAM_EVENT_AWARDS[team_key].keys())[-1]
                    and list(vals[1].keys())[-1].startswith("2025"),
                }
                for team_key, vals in longest_award_streak(False, False).items()
                if list(vals[1].keys())
            ],
            f,
            indent=None,
        )